In [ ]:
! pip install shapely geopandas pyproj

In [7]:
import shapely
import geopandas as gpd
import pandas as pd
import shapely.geometry.polygon as ply

def add_geo(r, geo):
    r['geometry'] = geo
    return r

def split_geos(r):
    if isinstance(r.geometry, ply.Polygon):
        return pd.DataFrame([r])
    df = pd.DataFrame([add_geo(r, geo) for geo in r.geometry])
    return df

def bufferz(c, rad, d):
    r = c.combine_first(d)
    r['geometry'] = c.geometry.buffer(rad)
    return r

def make_clusters(cities, dists, buffer_margin):
    dat = [(c, d) for _, d in dists.iterrows()
         for _, c in cities[cities.within(d.geometry)].iterrows() ]   

    dat = [(c, d.geometry.exterior.distance(c.geometry) - buffer_margin, d) for c, d in dat]
    dat = [bufferz(*t) for t in dat]
    df = gpd.GeoDataFrame(pd.DataFrame(dat), geometry='geometry', crs=3857).to_crs(4326)
    return df

In [5]:
cities = gpd.read_file('India_MNM/Cities_India/cities.shp')
districts = gpd.read_file('India_MNM/Districts+Demographics_India/Demographics_of_India.shp')
states = ['Uttar Pradesh', 'Jharkhand', 'Chhatisgarh']

# Split geos to make one row per polygon
dists = pd.concat([split_geos(r) for _,r in districts.iterrows()]).reset_index(drop=True)
dists = gpd.GeoDataFrame(dists, geometry='geometry', crs=4326)
dists = dists[dists.statename.isin(states)]

# Project to meter distance projection
dists = dists.to_crs(3857)
cities = cities.to_crs(3857)

In [53]:
dd = make_clusters(cities, dists, 2000)

In [55]:
dd.to_file('output/clusters.shp')